In [8]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [9]:
coach_features = pd.read_csv('data/generated_data/coach_features.csv')
stationary_probs_all = pd.read_csv('data/generated_data/stationary_probabilities.csv')
stationary_probs = stationary_probs_all[stationary_probs_all['Season'] >= 2003]
team_season_features = pd.read_csv('data/generated_data/team_season_features.csv')
additional_team_features = pd.read_csv('data/generated_data/additional_team_features.csv')
preseason_rankings = pd.read_csv('data/generated_data/preseason_rankings.csv')

In [10]:
coach_features.head()

,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
0,1102,2003,air-force,False,False,False,0.341,0,0,0,0
1,1103,2003,akron,False,False,False,0.462,0,0,0,0
2,1104,2003,alabama,False,True,False,0.660,4,0,0,0
3,1105,2003,alabama-am,False,False,False,0.518,0,0,0,0
4,1412,2003,alabama-birmingham,False,False,False,0.618,0,0,0,0


In [13]:
stationary_probs.head()

,Unnamed: 0,Season,TeamID,pi_i
5407,5407,2003,1102,0.002807
5408,5408,2003,1103,0.002684
5409,5409,2003,1104,0.004745
5410,5410,2003,1105,0.001239
5411,5411,2003,1106,0.001900


In [15]:
team_season_features.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement
0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,...,0.0,7.0,9.0,8.0,158.0,149.0,-9.0,160.0,151.0,-9.0
1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,...,1.0,1.0,13.0,2.0,182.0,198.0,16.0,163.0,187.0,24.0
2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,...,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0
3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,...,0.0,3.0,16.0,4.0,313.0,316.0,3.0,307.0,313.0,6.0
4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,...,0.0,4.0,11.0,5.0,248.0,236.0,-12.0,263.0,227.0,-36.0


In [16]:
additional_team_features.head()

,TeamID,Season,school,returning_min_pct,returning_score_pct,avg_height,avg_yr
0,1103,2022,akron,60.1,53.1,76,2.2
1,1104,2022,alabama,46.6,49.3,77,1.3
2,1412,2022,alabama-birmingham,60.0,63.2,77,3.1
3,1112,2022,arizona,48.3,45.9,80,1.5
4,1116,2022,arkansas,37.5,37.4,77,2.5


In [23]:
idx = ['Season', 'TeamID']
features_all = team_season_features.merge(stationary_probs, on=idx)\
.merge(coach_features, on=idx)\
.merge(additional_team_features, on=idx)\
.merge(preseason_rankings, on=idx)
features_all.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,...,sw16_car,ff_car,champ_car,school_y,returning_min_pct,returning_score_pct,avg_height,avg_yr,school,preseason_pts
0,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,...,0,0,0,alabama,80.5,78.0,76,1.8,alabama,1295
1,2003,1112,28,25,3,0.892857,85.214286,70.250000,14.964286,12.330225,...,12,5,1,arizona,98.4,99.0,77,1.6,arizona,1773
2,2003,1113,29,18,11,0.620690,75.965517,69.172414,6.793103,16.889865,...,0,0,0,arizona-state,73.2,66.6,77,1.6,arizona state,4
3,2003,1143,29,21,8,0.724138,74.482759,69.758621,4.724138,10.481170,...,2,0,0,california,47.7,53.2,76,1.7,california,3
4,2003,1153,28,17,11,0.607143,67.321429,61.500000,5.821429,12.889790,...,4,1,0,cincinnati,38.5,38.8,77,1.2,cincinnati,303


In [24]:
features_all[features_all['Season'] == 2022].sort_values(by='pi_i', ascending=False)[['school_y', 'pi_i']].head(50)

,school_y,pi_i
228,gonzaga,0.008481
233,kentucky,0.008245
221,arizona,0.007702
229,houston,0.007476
232,kansas,0.007394
223,auburn,0.007352
224,baylor,0.007277
241,purdue,0.007019
250,villanova,0.006919
247,texas-tech,0.006868


In [25]:
features_all.columns

Index(['Season', 'TeamID', 'total_games', 'num_wins', 'num_losses', 'win_pct',
       'avg_points_for', 'avg_points_against', 'avg_game_margin',
       'std_game_margin', 'non_home_wins', 'home_losses', 'close_wins',
       'close_losses', '3FGA_pg', '3FGM_pg', '3_pct', 'FTM_pg', 'FTA_pg',
       'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff', 'pythag_win_pct',
       'luck', 'Seed', 'wins_vs_tourney_teams', 'away_wins_vs_tourney_teams',
       'losses_vs_tourney_teams', 'losses_vs_non_tourney_teams',
       'games_vs_tourney_teams', 'final_net', 'prev_net', 'net_improvement',
       'final_pom', 'prev_pom', 'pom_improvement', 'Unnamed: 0', 'pi_i',
       'school_x', 'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'school_y', 'returning_min_pct', 'returning_score_pct', 'avg_height',
       'avg_yr', 'school', 'preseason_pts'],
      dtype='object')

In [29]:
features = features_all[['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for', 
              'avg_points_against', 'avg_game_margin', 'std_game_margin', 
              'non_home_wins', 'home_losses', 'close_wins', 'close_losses', 
              '3FGA_pg', '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 
              'off_eff', 'def_eff', 'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
              'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
              'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
              'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
              'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
              'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
              'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr', 'preseason_pts']]

In [30]:
features.head()

,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,...,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,avg_height,avg_yr,preseason_pts
0,2003,1104,Y10,28,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,...,0.660,4,0,0,0,80.5,78.0,76,1.8,1295
1,2003,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,...,0.738,24,12,5,1,98.4,99.0,77,1.6,1773
2,2003,1113,Z10,29,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,...,0.520,3,0,0,0,73.2,66.6,77,1.6,4
3,2003,1143,W08,29,0.724138,74.482759,69.758621,4.724138,10.481170,8.0,...,0.604,7,2,0,0,47.7,53.2,76,1.7,3
4,2003,1153,Z08,28,0.607143,67.321429,61.500000,5.821429,12.889790,5.0,...,0.737,13,4,1,0,38.5,38.8,77,1.2,303


In [ ]:
features['seed_num'] = features['Seed'].apply(lambda x: x.split())

## Tourney games

In [38]:
tourney_games_all = pd.read_csv('data/kaggle_data/MNCAATourneyCompactResults.csv')
tourney_games = tourney_games_all[tourney_games_all['Season'] >= 2003]
tourney_games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1136,2003,134,1421,92,1411,84,N,1
1137,2003,136,1112,80,1436,51,N,0
1138,2003,136,1113,84,1272,71,N,0
1139,2003,136,1141,79,1166,73,N,0
1140,2003,136,1143,76,1301,74,N,1


In [39]:
tourney_games['rand'] = [random.random() for i in range(len(tourney_games))]
tourney_games['teams'] = [(t1, t2) if r >= 0.5 else (t2, t1) for t1, t2, r in \
                          zip(tourney_games['WTeamID'], tourney_games['LTeamID'], tourney_games['rand'])]
tourney_games['t1_score_diff'] = [t1-t2 if r >= 0.5 else t2-t1 for t1, t2, r in \
                          zip(tourney_games['WScore'], tourney_games['LScore'], tourney_games['rand'])]
tourney_games[['team1','team2']] = pd.DataFrame(tourney_games['teams'].tolist(), index=tourney_games.index)

In [40]:
tourney_games

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,rand,teams,t1_score_diff,team1,team2
1136,2003,134,1421,92,1411,84,N,1,0.945225,"(1421, 1411)",8,1421,1411
1137,2003,136,1112,80,1436,51,N,0,0.441679,"(1436, 1112)",-29,1436,1112
1138,2003,136,1113,84,1272,71,N,0,0.365036,"(1272, 1113)",-13,1272,1113
1139,2003,136,1141,79,1166,73,N,0,0.274636,"(1166, 1141)",-6,1166,1141
1140,2003,136,1143,76,1301,74,N,1,0.985232,"(1143, 1301)",2,1143,1301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,2021,148,1211,85,1425,66,N,0,0.187989,"(1425, 1211)",-19,1425,1211
2313,2021,148,1417,51,1276,49,N,0,0.302397,"(1276, 1417)",-2,1276,1417
2314,2021,152,1124,78,1222,59,N,0,0.416098,"(1222, 1124)",-19,1222,1124
2315,2021,152,1211,93,1417,90,N,1,0.320814,"(1417, 1211)",-3,1417,1211


In [47]:
tourney_results = tourney_games[['Season', 'team1', 'team2', 't1_score_diff']]
tourney_results['is_t1_win'] = [diff > 0 for diff in tourney_results['t1_score_diff']]

In [48]:
tourney_results

,Season,team1,team2,t1_score_diff,is_t1_win
1136,2003,1421,1411,8,True
1137,2003,1436,1112,-29,False
1138,2003,1272,1113,-13,False
1139,2003,1166,1141,-6,False
1140,2003,1143,1301,2,True
...,...,...,...,...,...
2312,2021,1425,1211,-19,False
2313,2021,1276,1417,-2,False
2314,2021,1222,1124,-19,False
2315,2021,1417,1211,-3,False


In [49]:
matchup_features_all = tourney_results.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
.merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])
matchup_features_all.head()

,Season,team1,team2,t1_score_diff,is_t1_win,TeamID_1,Seed_1,total_games_1,win_pct_1,avg_points_for_1,...,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,avg_height_2,avg_yr_2,preseason_pts_2
0,2003,1272,1113,-13,False,1272,Z07,29,0.793103,74.517241,...,0.520,3,0,0,0,73.2,66.6,77,1.6,4
1,2003,1242,1113,32,True,1242,Z02,30,0.766667,81.333333,...,0.520,3,0,0,0,73.2,66.6,77,1.6,4
2,2003,1143,1301,2,True,1143,W08,29,0.724138,74.482759,...,0.605,3,0,0,0,57.7,53.3,78,1.6,41
3,2003,1143,1328,-9,False,1143,W08,29,0.724138,74.482759,...,0.633,10,3,1,0,63.0,58.4,77,1.4,1640
4,2003,1393,1328,16,True,1393,W03,29,0.827586,80.103448,...,0.633,10,3,1,0,63.0,58.4,77,1.4,1640


In [50]:
features.columns

Index(['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr',
       'preseason_pts'],
      dtype='object')

In [51]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr', 'preseason_pts']

In [52]:
for col in difference_cols:
    matchup_features_all['diff_'+col] = matchup_features_all[col+'_1'] - matchup_features_all[col+'_2']

In [53]:
pd.set_option("display.max_columns", None)
matchup_features_all.head()

,Season,team1,team2,t1_score_diff,is_t1_win,TeamID_1,Seed_1,total_games_1,win_pct_1,avg_points_for_1,avg_points_against_1,avg_game_margin_1,std_game_margin_1,non_home_wins_1,home_losses_1,close_wins_1,close_losses_1,3FGA_pg_1,3_pct_1,FTA_pg_1,FT_pct_1,TOV_pg_1,avg_pace_1,off_eff_1,def_eff_1,pythag_win_pct_1,luck_1,wins_vs_tourney_teams_1,away_wins_vs_tourney_teams_1,losses_vs_tourney_teams_1,losses_vs_non_tourney_teams_1,games_vs_tourney_teams_1,final_net_1,net_improvement_1,final_pom_1,pom_improvement_1,pi_i_1,is_ap_pre_top_5_1,is_ap_pre_top_15_1,is_ap_pre_top_25_1,coach_WL_car_1,tourneys_car_1,sw16_car_1,ff_car_1,champ_car_1,returning_min_pct_1,returning_score_pct_1,avg_height_1,avg_yr_1,preseason_pts_1,TeamID_2,Seed_2,total_games_2,win_pct_2,avg_points_for_2,avg_points_against_2,avg_game_margin_2,std_game_margin_2,non_home_wins_2,home_losses_2,close_wins_2,close_losses_2,3FGA_pg_2,3_pct_2,FTA_pg_2,FT_pct_2,TOV_pg_2,avg_pace_2,off_eff_2,def_eff_2,pythag_win_pct_2,luck_2,wins_vs_tourney_teams_2,away_wins_vs_tourney_teams_2,losses_vs_tourney_teams_2,losses_vs_non_tourney_teams_2,games_vs_tourney_teams_2,final_net_2,net_improvement_2,final_pom_2,pom_improvement_2,pi_i_2,is_ap_pre_top_5_2,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,avg_height_2,avg_yr_2,preseason_pts_2,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_avg_height,diff_avg_yr,diff_preseason_pts
0,2003,1272,1113,-13,False,1272,Z07,29,0.793103,74.517241,65.827586,8.689655,10.773862,9.0,2.0,2.0,4.0,20.068966,0.348797,22.896552,0.653614,13.793103,66.675789,111.152819,98.348954,0.848736,-0.055633,4.0,2.0,3.0,3.0,7.0,26.0,14.0,20.0,29.0,0.005375,False,False,False,0.721,6,3,1,0,NaN,NaN,76,NaN,2,1113,Z10,29,0.62069,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,0.317808,26.206897,0.669737,14.0,65.671136,115.334912,103.952243,0.786348,-0.165659,4.0,2.0,10.0,1.0,14.0,32.0,-2.0,32.0,-8.0,0.004979,False,False,False,0.520,3,0,0,0,73.2,66.6,77,1.6,4,0,0.172414,-1.448276,-3.344828,1.896552,-6.116003,3.0,-1.0,-1.0,1.0,7.482759,0.030989,-3.310345,-0.016122,-0.206897,1.004653,-4.182093,-5.603289,0.062388,0.110026,0.0,0.0,-7.0,2.0,-7.0,-6.0,16.0,-12.0,37.0,0.000396,0.201,3,3,1,0,NaN,NaN,-1,NaN,-2
1,2003,1242,1113,32,True,1242,Z02,30,0.766667,81.333333,66.366667,14.966667,17.827850,10.0,1.0,1.0,1.0,14.133333,0.339623,24.133333,0.665746,14.900000,70.337280,115.294347,94.655205,0.944210,-0.177543,9.0,3.0,6.0,1.0,15.0,6.0,5.0,4.0,2.0,0.007841,True,False,False,0.805,14,9,4,0,62.2,59.6,77,1.5,1715,1113,Z10,29,0.62069,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,0.317808,26.206897,0.669737,14.0,65.671136,115.334912,103.952243,0.786348,-0.165659,4.0,2.0,10.0,1.0,14.0,32.0,-2.0,32.0,-8.0,0.004979,False,False,False,0.520,3,0,0,0,73.2,66.6,77,1.6,4,1,0.145977,5.367816,-2.805747,8.173563,0.937985,4.0,-2.0,-2.0,-2.0,1.547126,0.021814,-2.073563,-0.003991,0.900000,4.666144,-0.040565,-9.297038,0.157862,-0.011885,5.0,1.0,-4.0,0.0,1.0,-26.0,7.0,-28.0,10.0,0.002862,0.285,11,9,4,0,-11.0,-7.0,0,-0.1,1711
2,2003,1143,1301,2,True,1143,W08,29,0.724138,74.482759,69.758621,4.724138,10.481170,8.0,1.0,2.0,2.0,17.034483,0.376518,19.517241,0.685512,14.172414,66.586777,110.701125,103.757099,0.713302,0.010836,4.0,1.0,5.0,3.0,9.0,33.0,-7.0,40.0,-13.0,0.004612,False,F

In [58]:
diff_cols = [col for col in matchup_features_all.columns if 'diff' in col]
matchup_cols = ['TeamID_1', 'TeamID_2', 'Seed_1', 'Seed_2'] + diff_cols

In [62]:
matchup_features_basic = matchup_features_all[matchup_cols]

In [63]:
matchup_features_basic.to_csv('data/model_data/matchup_features.csv', index=False)

In [64]:
matchup_features_basic.head()

,TeamID_1,TeamID_2,Seed_1,Seed_2,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_avg_height,diff_avg_yr,diff_preseason_pts
0,1272,1113,Z07,Z10,-13,0,0.172414,-1.448276,-3.344828,1.896552,-6.116003,3.0,-1.0,-1.0,1.0,7.482759,0.030989,-3.310345,-0.016122,-0.206897,1.004653,-4.182093,-5.603289,0.062388,0.110026,0.0,0.0,-7.0,2.0,-7.0,-6.0,16.0,-12.0,37.0,0.000396,0.201,3,3,1,0,NaN,NaN,-1,NaN,-2
1,1242,1113,Z02,Z10,32,1,0.145977,5.367816,-2.805747,8.173563,0.937985,4.0,-2.0,-2.0,-2.0,1.547126,0.021814,-2.073563,-0.003991,0.900000,4.666144,-0.040565,-9.297038,0.157862,-0.011885,5.0,1.0,-4.0,0.0,1.0,-26.0,7.0,-28.0,10.0,0.002862,0.285,11,9,4,0,-11.0,-7.0,0,-0.1,1711
2,1143,1301,W08,W09,2,-1,0.124138,2.082759,1.758621,0.324138,-5.455370,3.0,-2.0,0.0,0.0,-5.465517,0.022444,-0.949425,-0.084846,-0.027586,2.833959,-2.167815,-2.687209,0.008112,0.116026,2.0,0.0,-2.0,-2.0,0.0,-20.0,-20.0,-8.0,-26.0,0.000024,-0.001,4,2,0,0,-10.0,-0.1,-2,0.1,-38
3,1143,1328,W08,W01,-9,-1,-0.075862,3.316092,9.591954,-6.275862,-3.906324,-1.0,0.0,-3.0,1.0,-1.932184,-0.017155,0.917241,-0.022373,2.372414,5.257924,-3.829212,7.029510,-0.198479,0.122617,-4.0,-2.0,-1.0,3.0,-5.0,30.0,-10.0,31.0,-15.0,-0.002071,-0.029,-3,-1,-1,0,-15.3,-5.2,-1,0.3,-1637
4,1393,1328,W03,W01,16,-1,0.027586,8.936782,9.729885,-0.793103,-0.450743,-2.0,-1.0,-1.0,0.0,-3.104598,-0.063238,5.020690,-0.014455,1.820690,7.240673,1.150369,5.033997,-0.042346,0.069932,-3.0,-1.0,-2.0,1.0,-5.0,6.0,3.0,6.0,2.0,-0.000715,0.110,12,9,2,1,-16.3,-17.1,2,-0.4,-1561
